In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import torch

In [ ]:
!pip install torch_geometric -q
!pip install torcheval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 24.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 5.5 MB/s eta 0:00:00


In [ ]:
edges = pd.read_csv('/content/drive/MyDrive/TAI/III/gnn/data/twitch_gamers/large_twitch_edges.csv')
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6797557 entries, 0 to 6797556
Data columns (total 2 columns):
 #   Column        Dtype
---  ------        -----
 0   numeric_id_1  int64
 1   numeric_id_2  int64
dtypes: int64(2)
memory usage: 103.7 MB


In [ ]:
edges

,numeric_id_1,numeric_id_2
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118
...,...,...
6797552,97507,29359
6797553,71175,12020
6797554,151702,128281
6797555,118034,38021


In [ ]:
nodes = pd.read_csv('/content/drive/MyDrive/TAI/III/gnn/data/twitch_gamers/large_twitch_features.csv')
nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168114 entries, 0 to 168113
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   views         168114 non-null  int64 
 1   mature        168114 non-null  int64 
 2   life_time     168114 non-null  int64 
 3   created_at    168114 non-null  object
 4   updated_at    168114 non-null  object
 5   numeric_id    168114 non-null  int64 
 6   dead_account  168114 non-null  int64 
 7   language      168114 non-null  object
 8   affiliate     168114 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 11.5+ MB


In [ ]:
nodes.created_at = pd.to_datetime(nodes.created_at)
nodes.updated_at = pd.to_datetime(nodes.updated_at)
nodes['created_at_year'] = nodes['created_at'].dt.year
nodes['created_at_month'] = nodes['created_at'].dt.month
nodes['created_at_day'] = nodes['created_at'].dt.day
nodes['updated_at_year'] = nodes['updated_at'].dt.year
nodes['updated_at_month'] = nodes['updated_at'].dt.month
nodes['updated_at_day'] = nodes['updated_at'].dt.day
nodes = pd.get_dummies(nodes, columns=['language'])
nodes = nodes.drop(columns=['created_at', 'updated_at'])

# Minmax Normalization

In [ ]:
nodes['views'] = (nodes['views'] - nodes['views'].min())/ (nodes['views'].max() - nodes['views'].min())
nodes['life_time'] = (nodes['life_time'] - nodes['life_time'].min())/ (nodes['life_time'].max() - nodes['life_time'].min())
nodes['created_at_year'] = (nodes['created_at_year'] - nodes['created_at_year'].min()) / (nodes['created_at_year'].max() - nodes['created_at_year'].min())
nodes['updated_at_year'] = (nodes['updated_at_year'] - nodes['updated_at_year'].min()) / (nodes['updated_at_year'].max() - nodes['updated_at_year'].min())
nodes['created_at_month'] = nodes['created_at_month']  / 12
nodes['updated_at_month'] = nodes['updated_at_month'] / 12
nodes['created_at_day'] = nodes['created_at_day']  / 31
nodes['updated_at_day'] = nodes['updated_at_day'] / 31

In [ ]:
nodes = nodes.set_index(['numeric_id'])

In [ ]:
nodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168114 entries, 0 to 168113
Data columns (total 32 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   views             168114 non-null  float64
 1   mature            168114 non-null  int64  
 2   life_time         168114 non-null  float64
 3   dead_account      168114 non-null  int64  
 4   affiliate         168114 non-null  int64  
 5   created_at_year   168114 non-null  float64
 6   created_at_month  168114 non-null  float64
 7   created_at_day    168114 non-null  float64
 8   updated_at_year   168114 non-null  float64
 9   updated_at_month  168114 non-null  float64
 10  updated_at_day    168114 non-null  float64
 11  language_CS       168114 non-null  uint8  
 12  language_DA       168114 non-null  uint8  
 13  language_DE       168114 non-null  uint8  
 14  language_EN       168114 non-null  uint8  
 15  language_ES       168114 non-null  uint8  
 16  language_FI       16

In [ ]:
nodes

,views,mature,life_time,dead_account,affiliate,created_at_year,created_at_month,created_at_day,updated_at_year,updated_at_month,...,language_NL,language_NO,language_OTHER,language_PL,language_PT,language_RU,language_SV,language_TH,language_TR,language_ZH
numeric_id,,,,,,,,,,,,,,,,,,,,,
0,0.000020,1,0.230843,0,1,0.818182,0.166667,0.516129,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
1,0.000001,0,0.647711,0,0,0.363636,0.416667,0.612903,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
2,0.000995,1,0.756145,0,1,0.272727,0.166667,0.870968,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
3,0.000001,0,0.321205,0,0,0.727273,0.083333,0.838710,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
4,0.000006,0,0.427229,0,0,0.545455,0.916667,0.709677,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168109,0.000013,0,0.192530,0,0,0.818182,0.583333,0.645161,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
168110,0.000011,1,0.498554,0,0,0.545455,0.083333,1.000000,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0
168111,0.000009,0,0.430361,0,1,0.545455,0.916667,0.258065,1.0,0.833333,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
target = ['life_time']
features = nodes.columns.difference(target, sort=False)

In [ ]:
print(target)

['life_time']


In [ ]:
from sklearn.preprocessing import StandardScaler
import torch

X = np.asarray(nodes[features].values)
y = np.asarray(nodes[target].values)


In [ ]:
G = nx.from_pandas_edgelist(edges, source='numeric_id_1', target='numeric_id_2')

In [ ]:
G.number_of_nodes()

168114

In [ ]:
# pos = nx.spring_layout(G, seed=42)
# nx.draw_networkx_edges(G, pos, alpha=0.2)
# nx.draw_networkx_nodes(G, pos, nodelist=G.nodes(), node_size=1)

In [ ]:
adj = nx.to_scipy_sparse_array(G).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64))
col = torch.from_numpy(adj.col.astype(np.int64))
edge_index = torch.stack([row, col], dim=0)

In [ ]:
print(X.shape)
print(y.shape)
print(edge_index.shape)

(168114, 31)
(168114, 1)
torch.Size([2, 13595114])


In [ ]:
len(y[0])

1

In [ ]:

from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader

class CustomDataset(InMemoryDataset):
    def __init__(self, transform=None):
        super(CustomDataset, self).__init__('.', transform, None, None)

        data = Data(edge_index=edge_index)
        data.x = torch.from_numpy(X).type(torch.float32)
        data.y = torch.from_numpy(y).type(torch.float32)
        data.num_classes = len(y[0])

        # splitting the data into train, validation and test
        X_train, X_test, y_train, y_test = train_test_split(pd.Series(X[:,0]),pd.Series(y[:,0]),test_size=0.20,random_state=42)

        # create train and test masks for data
        train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        train_mask[X_train.index] = True
        test_mask[X_test.index] = True
        data['train_mask'] = train_mask
        data['test_mask'] = test_mask

        self.data, self.slices = self.collate([data])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# GCN model with 2 layers
class Net(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Net, self).__init__()
        self.conv1 = GCNConv(input_dim, 16)
        self.conv2 = GCNConv(16, output_dim)
        self.linear = nn.Linear(output_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.linear(x)
        return x

In [ ]:
from torch import optim

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = CustomDataset()
data = dataset[0].to(device)

# Define the GNN model
input_dim = dataset.num_node_features  # Number of input features
output_dim = data.num_classes # Number of output classes
model = Net(input_dim, output_dim).to(device)

# Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
# criterion = nn.NLLLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from torcheval.metrics.functional import multiclass_auroc
def evaluate(model, data, data_mask):
    model.eval()
    preds = model(data)
    trues = data.y[data_mask]
    mse = criterion(preds[data_mask], trues)
    # correct = torch.sum(torch.argmax(preds, dim=1)[data_mask] == trues)
    # acc = correct / len(trues)
    # roc = multiclass_auroc(preds[data_mask], trues, num_classes = len(preds[0]))
    return mse

In [ ]:
mse = evaluate(model, data, data.test_mask)
print(f'Accuracy: {mse:.4f}')

Accuracy: 0.1175


In [ ]:
num_epochs = 20
model.train()
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0

    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(data)

    # Compute loss
    loss = criterion(outputs[data.train_mask], data.y[data.train_mask])

    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    epoch_loss = running_loss

    print(f'epoch {epoch+1}/{num_epochs} - mse: {epoch_loss:.3f}')


epoch 1/20 - mse: 0.118
epoch 2/20 - mse: 0.079
epoch 3/20 - mse: 0.053
epoch 4/20 - mse: 0.044
epoch 5/20 - mse: 0.049
epoch 6/20 - mse: 0.055
epoch 7/20 - mse: 0.057
epoch 8/20 - mse: 0.052
epoch 9/20 - mse: 0.047
epoch 10/20 - mse: 0.042
epoch 11/20 - mse: 0.040
epoch 12/20 - mse: 0.038
epoch 13/20 - mse: 0.038
epoch 14/20 - mse: 0.037
epoch 15/20 - mse: 0.037
epoch 16/20 - mse: 0.037
epoch 17/20 - mse: 0.037
epoch 18/20 - mse: 0.036
epoch 19/20 - mse: 0.036
epoch 20/20 - mse: 0.035


In [ ]:
mse = evaluate(model, data, data.test_mask)
print(f'Accuracy: {mse:.4f}')

Accuracy: 0.0348
